In [ ]:
# 코드 작성자 : 조성환
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random

# 시각화의 선명도를 높임
%config InlineBackend.figure_format = "retina"

# 한글설정
sns.set_theme(style='white', rc={'figure.figsize':(12,5)})
plt.rc('font', family='Malgun Gothic') # 윈도우즈
plt.rc('axes', unicode_minus=False) # 축의 - 깨짐 방지

# 다른 폰트를 기본 폰트로 하려면
from matplotlib.font_manager import FontProperties
font_path = 'C:/Windows/Fonts/CAT.ttf'
font_prop = FontProperties(fname=font_path, size=30)
# plt.rc('font', family=font_prop.get_name())

# warning(경고) 안보이게
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 1) 파일 로드
raw = pd.read_csv('C:/taro/1차_프로젝트/카드데이터/수원시팔달구행정동명백업.csv')

# 2) 날짜별 평균매출액 계산 (총매출/총건수)
daily = (
    raw.groupby("기준년월일", as_index=False)[["매출금액", "매출건수"]]
       .sum()
)
daily["평균매출"] = (daily["매출금액"] / daily["매출건수"]).round().astype(int)

# 3) 메뉴/가중치
menus = {"아메리카노": 2000, "라떼": 3000, "카푸치노": 3000}
menu_weights = {"아메리카노": 0.5, "라떼": 0.3, "카푸치노": 0.2}

menu_list = list(menus.keys())
weights = list(menu_weights.values())

def make_mixed_basket(target_amount: int) -> dict:
    """target_amount(평균매출액 1건) 안에서 여러 메뉴가 섞이도록 구성"""
    counts = {m: 0 for m in menus}
    total = 0

    # 1잔도 못 사는 경우 방지(데이터에 따라 거의 없겠지만 안전장치)
    min_price = min(menus.values())
    if target_amount < min_price:
        return counts | {"총금액": 0}

    while True:
        m = random.choices(menu_list, weights=weights)[0]
        price = menus[m]
        if total + price > target_amount:
            break
        counts[m] += 1
        total += price

    counts["총금액"] = total
    return counts

# 4) 날짜별 한 줄(통합) 프레임 만들기
basket_rows = []
for _, r in daily.iterrows():
    basket = make_mixed_basket(int(r["평균매출"]))
    basket_rows.append({
        "기준년월일": int(r["기준년월일"]),
        "평균매출": int(r["평균매출"]),
        **basket
    })

result = pd.DataFrame(basket_rows)

# 보기 좋게 날짜 형식 변환(옵션)
result["기준년월일"] = pd.to_datetime(result["기준년월일"].astype(str), format="%Y%m%d")

# 원하는 컬럼 순서로 정리
result = result[["기준년월일", "평균매출", "아메리카노", "라떼", "카푸치노", "총금액"]]

result

,기준년월일,평균매출,아메리카노,라떼,카푸치노,총금액
0,2023-01-01,13724,1,1,2,11000
1,2023-01-02,10342,1,2,0,8000
2,2023-01-03,10420,4,0,0,8000
3,2023-01-04,10684,2,0,2,10000
4,2023-01-05,10930,2,2,0,10000
...,...,...,...,...,...,...
1030,2025-10-27,8141,3,0,0,6000
1031,2025-10-28,8940,2,1,0,7000
1032,2025-10-29,8704,0,0,2,6000
1033,2025-10-30,9079,2,0,1,7000


In [3]:
# 파일 저장
result.to_csv(
    "C:/taro/1차_프로젝트/카드데이터/메뉴랜덤데이터.csv",
    index=False,
    encoding="utf-8-sig"
)

In [4]:
# 생성 파일 확인
df = pd.read_csv('C:/taro/1차_프로젝트/카드데이터/메뉴랜덤데이터.csv',
                       encoding='utf-8')
df

,기준년월일,평균매출,아메리카노,라떼,카푸치노,총금액
0,2023-01-01,13724,1,1,2,11000
1,2023-01-02,10342,1,2,0,8000
2,2023-01-03,10420,4,0,0,8000
3,2023-01-04,10684,2,0,2,10000
4,2023-01-05,10930,2,2,0,10000
...,...,...,...,...,...,...
1030,2025-10-27,8141,3,0,0,6000
1031,2025-10-28,8940,2,1,0,7000
1032,2025-10-29,8704,0,0,2,6000
1033,2025-10-30,9079,2,0,1,7000
